# Setup packages

In [1]:
# Install required packages
!pip3 install pycuda
!pip3 install nvidia-dllogger
!pip3 install nvidia-pyindex
!pip3 install nvidia-tensorrt==8.4.3.1
# !pip3 install torch-tensorrt -f https://github.com/NVIDIA/Torch-TensorRT/releases
!pip3 install numpy scipy librosa unidecode inflect librosa colored
!pip3 install onnxruntime onnx_graphsurgeon
!pip3 install parallel_wavegan

!apt-get update
!apt-get install -y libsndfile1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 1.7 MB 37.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 101.9 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 123.5 MB/s eta 0:00:01
  Created wheel for pycuda: filename=pycuda-2022.1-cp38-cp38-linux_x86_64.whl size=660248 sha256=49fa1d7ed40de7163fdd7f371620ee17aff0fb3c82f2ec6420380845a8c2e3aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-qn26lzkv/wheels/55/18/86/7aeb1c40b261b96a14f102cfd359298936d56a364a9bc6ad4c
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=2f200ef13fd94fd78607b71436b8b4a89313e3790c9b94fd12ef304d2e796919
  Stored in directory: /tmp/pip-ephem-wheel-cache-qn26lzkv/wheels/29/70/ac/4beedc24ff54ecfa2b53653ebbe7cdab8ec45ce9af2ee728c5
Successfully built p

# Download models

### Tacotron2

In [2]:
# Download tacotron2 amp checkpoint from NGC
!mkdir -p checkpoints
!wget -nc --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/dle/tacotron2__pyt_ckpt/versions/19.12.0_amp/zip -O tacotron2__pyt_ckpt_19.12.0_amp.zip

# Unzip and cp to ./checkpoints
!unzip -o tacotron2__pyt_ckpt_19.12.0_amp.zip
!mv nvidia_tacotron2pyt_fp16.pt ./checkpoints
!rm tacotron2__pyt_ckpt_19.12.0_amp.zip

--2022-10-18 00:05:00--  https://api.ngc.nvidia.com/v2/models/nvidia/dle/tacotron2__pyt_ckpt/versions/19.12.0_amp/zip
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 35.162.223.184, 35.82.109.218
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|35.162.223.184|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://prod-model-registry-ngc-bucket.s3.us-west-2.amazonaws.com/org/nvidia/team/dle/models/tacotron2__pyt_ckpt/versions/19.12.0_amp/files.zip?response-content-disposition=attachment%3B%20filename%3D%22files.zip%22&response-content-type=application%2Fzip&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEEcaCXVzLXdlc3QtMiJHMEUCIEa8sNyTukf6ndQxsC9G79tKtueChGNwSCj9tgr00IMWAiEA4%2Fwk%2ByLJ0v5jB2s%2Fmx%2Bb5kcqGKOe5QGm6QTVp58lz8oqzAQIIBAEGgw3ODkzNjMxMzUwMjciDB%2BMMoEg55xNwHw2HCqpBDsXVefFwdmahyIF%2FLHtRvWpRXMtn2glPO11aYL1%2FuISSp7VYtnAQfY0A7SZt9FWgHbcu9im2X4gdRjxqCLnxkPL4gI6ENPnRyHcEiS%2F5%2BBNJcr%2BSfQdq19DTO5QbMckOnK5TBZep%2BmXorXVtj%2FMeSdjy8U%2By6%2BaBhBC3IuN9eI

### Waveglow

You might need to setup **NGC** CLI in advance.

In [3]:
# Download waveglow amp checkpoint
!mkdir -p checkpoints
# !ngc registry model download-version "nvidia/waveglow256pyt_fp16:2"  # or download the ckpt directly from https://ngc.nvidia.com/models/nvidia:waveglow256pyt_fp16

# mv to ./checkpoints
!mv waveglow256pyt_fp16_v2/waveglow_1076430_14000_amp checkpoints/nvidia_waveglow256pyt_fp16.pt
!rmdir waveglow256pyt_fp16_v2

### ParallelWaveGan

Referring https://github.com/kan-bayashi/ParallelWaveGAN

In [4]:
# Download parallelwavegan checkpoint
from parallel_wavegan.utils import download_pretrained_model, PRETRAINED_MODEL_LIST
print(PRETRAINED_MODEL_LIST.keys())
download_pretrained_model("ljspeech_parallel_wavegan.v1.long","checkpoints")

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['ljspeech_parallel_wavegan.v1', 'ljspeech_parallel_wavegan.v1.long', 'ljspeech_parallel_wavegan.v1.no_limit', 'ljspeech_parallel_wavegan.v3', 'ljspeech_melgan.v1', 'ljspeech_melgan.v1.long', 'ljspeech_melgan.v3', 'ljspeech_melgan.v3.long', 'ljspeech_full_band_melgan.v2', 'ljspeech_multi_band_melgan.v2', 'ljspeech_hifigan.v1', 'ljspeech_style_melgan.v1', 'jsut_parallel_wavegan.v1', 'jsut_multi_band_melgan.v2', 'jsut_hifigan.v1', 'jsut_style_melgan.v1', 'csmsc_parallel_wavegan.v1', 'csmsc_multi_band_melgan.v2', 'csmsc_hifigan.v1', 'csmsc_style_melgan.v1', 'arctic_slt_parallel_wavegan.v1', 'jnas_parallel_wavegan.v1', 'vctk_parallel_wavegan.v1', 'vctk_parallel_wavegan.v1.long', 'vctk_multi_band_melgan.v2', 'vctk_hifigan.v1', 'vctk_style_melgan.v1', 'libritts_parallel_wavegan.v1', 'libritts_parallel_wavegan.v1.long', 'libritts_multi_band_melgan.v2', 'libritts_hifigan.v1', 'libritts_style_melgan.v1', 'kss_parallel_wavegan.v1', 'hui_acg_hokuspokus_parallel_wavegan.v1', 'ruslan_para

Downloading...
From: https://drive.google.com/uc?id=1A9TsrD9fHxFviJVFjCk5W6lkzWXwhftv
To: /workspace/tensorRT/pwavegan/checkpoints/ljspeech_parallel_wavegan.v1.long.tar.gz
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15.6M/15.6M [00:00<00:00, 36.3MB/s]


'checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl'

# Simple inference test from the ckpts

### Tacotron2 + WaveGlow

In [5]:
## Simple Inference FP16 test(Tacotron2 + Waveglow)
# Define config parser
class SoftDict:
	def __init__(self, user_dict):
		self._user_dict = user_dict
		self._parse()

	def _parse(self):
		for key in self._user_dict.keys():
			value = self._user_dict[key]
			if type(value) == dict:
				value = SoftDict(value)
			setattr(self, key, value)

# Get Tacotron2 from the checkpoint
import torch, models
tacotron2_ckpt = torch.load("checkpoints/nvidia_tacotron2pyt_fp16.pt")
config = tacotron2_ckpt['config']
config['fp16_run'] = True
args = SoftDict(config)
state_dict = tacotron2_ckpt['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("module.","")] = state_dict.pop(key)    
    
model_config = models.get_model_config("Tacotron2", args)
tacotron2 = models.get_model("Tacotron2", model_config, False, forward_is_infer=True)
tacotron2.load_state_dict(state_dict)
tacotron2.eval()

# Get Waveglow from the checkpoint
# waveglow_ckpt = torch.load("checkpoints/waveglowpyt_fp16_20210323.pt")
waveglow_ckpt = torch.load("checkpoints/nvidia_waveglow256pyt_fp16.pt")
config = waveglow_ckpt['config']
args = SoftDict(config)
state_dict = waveglow_ckpt['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("module.","")] = state_dict.pop(key)  

waveglow = models.get_model("WaveGlow", config, False, forward_is_infer=True)
waveglow.load_state_dict(state_dict)
waveglow.eval()

WaveGlow__forward_is_infer(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
        (1): 

In [9]:
# Setup Sample text
text = "Hello my name is Woojin. Nice to meet you."
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, sequence_lengths = utils.prepare_input_sequence([text])

# Infer
with torch.no_grad():
    rate = 22050
    mel, mel_lengths, alignments = tacotron2(sequences, sequence_lengths)
    audio = waveglow.infer(mel)
    audio_numpy = audio[0].data.cpu().numpy()
#     from scipy.io.wavfile import write
#     write("audio.wav", rate, audio_numpy)

from IPython.display import Audio
display(Audio(audio_numpy, rate=rate))

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


### Tacotron2 + ParallelWaveGan

Mel basis might be different between pretrained Tacotron2(from NGC) and ParallelWaveGan(from https://github.com/kan-bayashi/ParallelWaveGAN), which could causes lots of noise to the synthesis result. Just to check whether inference-flow works or not.

In [10]:
# Define config parser
class SoftDict:
	def __init__(self, user_dict):
		self._user_dict = user_dict
		self._parse()

	def _parse(self):
		for key in self._user_dict.keys():
			value = self._user_dict[key]
			if type(value) == dict:
				value = SoftDict(value)
			setattr(self, key, value)

# Get Tacotron2 from the checkpoint
import torch, models
tacotron2_ckpt = torch.load("checkpoints/nvidia_tacotron2pyt_fp16.pt")
config = tacotron2_ckpt['config']
config['fp16_run'] = True
args = SoftDict(config)
state_dict = tacotron2_ckpt['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace("module.","")] = state_dict.pop(key)    
    
model_config = models.get_model_config("Tacotron2", args)
tacotron2 = models.get_model("Tacotron2", model_config, False, forward_is_infer=True)
tacotron2.load_state_dict(state_dict)
tacotron2.eval()

# Get ParallelWaveGan Generator
import torch
import yaml
from parallel_wavegan.models import ParallelWaveGANGenerator

pwg_ckpt=torch.load("checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl")
state_dict = pwg_ckpt['model']['generator']
with open("checkpoints/ljspeech_parallel_wavegan.v1.long/config.yml") as f:
    cfg = yaml.safe_load(f)
generator_cfg = cfg['generator_params']

parallelwavegan = ParallelWaveGANGenerator(**generator_cfg)
parallelwavegan.load_state_dict(state_dict)
parallelwavegan.half()
parallelwavegan.cuda().eval()

ParallelWaveGANGenerator(
  (first_conv): Conv1d1x1(1, 64, kernel_size=(1,), stride=(1,))
  (upsample_net): ConvInUpsampleNetwork(
    (conv_in): Conv1d(80, 80, kernel_size=(5,), stride=(1,), bias=False)
    (upsample): UpsampleNetwork(
      (up_layers): ModuleList(
        (0): Stretch2d()
        (1): Conv2d(1, 1, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4), bias=False)
        (2): Stretch2d()
        (3): Conv2d(1, 1, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4), bias=False)
        (4): Stretch2d()
        (5): Conv2d(1, 1, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4), bias=False)
        (6): Stretch2d()
        (7): Conv2d(1, 1, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4), bias=False)
      )
    )
  )
  (conv_layers): ModuleList(
    (0): WaveNetResidualBlock(
      (conv): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv1x1_aux): Conv1d1x1(80, 128, kernel_size=(1,), stride=(1,), bias=False)
      (conv1x1_out): Conv1d1x1(64, 64, k

In [11]:
import numpy as np
from tacotron2_common.audio_processing import dynamic_range_decompression
from parallel_wavegan.utils import read_hdf5

# Setup Sample text
text = "Hello my name is Woojin. Nice to meet you."
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, sequence_lengths = utils.prepare_input_sequence([text])

# Infer
with torch.no_grad():
    # Generate Spectrogram
    mel, mel_lengths, alignments = tacotron2(sequences, sequence_lengths)
    
    # Decompress and log10 the output
    decompressed = dynamic_range_decompression(mel)
    decompressed_log10 = np.log10(decompressed.cpu()).cuda()
    stats_path = "checkpoints/ljspeech_parallel_wavegan.v1.long/stats.h5"
    mu = read_hdf5(stats_path, "mean")
    sigma = read_hdf5(stats_path, "scale")
    decompressed_log10_norm = (decompressed_log10 - torch.from_numpy(mu).view(1, -1, 1).cuda()) / torch.from_numpy(sigma).view(1, -1, 1).cuda()

    # Prepare for inputs
    rate = 22050
    upsample_factor=256
    c = decompressed_log10_norm.squeeze(0).transpose(0,1)
    x = torch.randn(1, 1, len(c) * upsample_factor).to("cuda")
    c = c.transpose(1,0).unsqueeze(0)
    c = torch.nn.ReplicationPad1d(2)(c)

    x = x.half()
    c = c.half()
    
    # Synthesis
    pred = parallelwavegan(x,c).squeeze(0).transpose(1,0)

from IPython.display import display, Audio
display(Audio(pred.view(-1).cpu().detach().numpy(), rate=rate))

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
